# 302.0 Conexión a una base de datos con Python

Existen varias opciones para conectarse a una base de datos desde python, en particular para una conexión a postgreSQL las principales opciones son el driver psycopg2 o la librería sqlAlchemy.

```{admonition} Recursos

- Conjunto de datos para el ejercicio: {download}`Datos Call Center<datos_call_center.zip>`

- Jupyter Notebook : {download}`Conexión Python RDS<302.0_conexion_python_RDS.ipynb>`

```

In [1]:
#conda create --name NOMBRE python=3.10

## Conexión usando el driver psycopg2

Dentro de las ventajas de esta librería se encuentran sus capacidades multihilo, que le permiten manejar un gran número de escrituras y lecturas de manera concurrente. 

El comando básico para realizar la instalación mediante el instalador de paquetes [pip](https://pypi.org/project/pip/) de python es 

`$ pip install psycopg2-binary`

Tenga en cuenta los pre-requisitos para esta librería

- Versión de Python 3.6 a 3.10
- Versión de postgreSQL 7.4 to 14
- Cliente de postgreSQL 9.1 o superior

```{note}
Información adicional de instalación está disponible en  https://www.psycopg.org/docs/install.html#install-from-source
```

**Notas:** 
**Asegure la compatibilidad del entorno de instalación usando conda, venv o docker**
**Versión Python 3.10 para evitar errores de conexión por contraseña**

https://docs.conda.io/en/latest/miniconda.html

In [2]:
# Librerías requeridas para la conexión y muestra de información
import psycopg2 as ps
import pandas as pd

In [2]:
#parámetros de conexión AWS, usar variables de entorno
host = "database-1.cloyu6seeky2.us-east-1.rds.amazonaws.com"
port = '5432'
user = 'postgres'
password = 'unisabana2024#'
database = 'callcenterdb' #verificar la existencia de la base datos

In [3]:
# #parámetros de conexión Local, usar variables de entorno
# host = "localhost"
# port = '5432'
# user = 'user'
# password = 'password'
# database = 'mydatabase' #verificar la existencia de la base datos

In [4]:
# Conectar a la base de datos

try:
    connps = ps.connect(host=host,database=database,user=user,password=password,port=port)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

Connected!


In [6]:
# Lectura de la tabla call_data de la base de datos
call_data = pd.read_sql('SELECT * FROM calldata', connps)

C:\Users\rodol\AppData\Local\Temp\ipykernel_38616\1263083436.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  call_data = pd.read_sql('SELECT * FROM calldata', connps)


In [7]:
#muestra de la información
call_data.head()

,YR_MO,CALL_DATE,AGENT_KEY,CALLS,HANDLE_TIME,CALL_REGEN,CALLS_WITH_OFFER,CALLS_WITH_ACCEPT,CALLS_OFFER_APPLIED,TRANSFERS
0,201706,2017-06-01,12465,34,25710,5,31,22,19,2
1,201706,2017-06-01,12466,35,25039,4,34,29,26,2
2,201706,2017-06-01,12467,38,24436,6,36,32,28,3
3,201706,2017-06-01,12468,38,24923,5,37,32,25,2
4,201706,2017-06-01,12469,34,24371,4,32,25,22,2


## Conexión a la base de datos usando SQLAlchemy
Para la conexión usando SQLAlchemy se usaran los mismos parámetros cambiando la sintaxis e importando las librerias de SQLAlchemy

In [8]:
from sqlalchemy import create_engine

In [9]:
connAlchemy = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = connAlchemy.raw_connection()
cur = conn.cursor()

In [10]:
call_data2 = pd.read_sql('SELECT * FROM calldata', connAlchemy)

In [11]:
call_data2.head()

,YR_MO,CALL_DATE,AGENT_KEY,CALLS,HANDLE_TIME,CALL_REGEN,CALLS_WITH_OFFER,CALLS_WITH_ACCEPT,CALLS_OFFER_APPLIED,TRANSFERS
0,201706,2017-06-01,12465,34,25710,5,31,22,19,2
1,201706,2017-06-01,12466,35,25039,4,34,29,26,2
2,201706,2017-06-01,12467,38,24436,6,36,32,28,3
3,201706,2017-06-01,12468,38,24923,5,37,32,25,2
4,201706,2017-06-01,12469,34,24371,4,32,25,22,2


```{note}
Compare los tiempos de conexión y lectura de información de las dos librerías, que puede concluir de esta prueba.
```

## Carga masiva de datos 
En este proceso realizaremos una carga de datos realizando previamente una transformación de la información.

In [40]:
# librerias
import re
import io
from unicodedata import normalize

En algunos casos es necesario realizar algunas transformaciones básicas a la información recibida antes de realizar el proceso de carga a la base de datos

En este artículo pueden encontrar información adicional acerca de la función normalize y el parámetro NFC. https://towardsdatascience.com/what-on-earth-is-unicode-normalization-56c005c55ad0

In [41]:
# función de transformación del texto  para los nombres de las columnas en sql
def formatcolumn(text,flag = True):
    text = text.lower()
    text = re.sub('<.*?>', '', text)
    text = re.sub(':.*?:', '', text)
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    text = normalize( 'NFC', text)
    if flag:
        text = re.sub('[^a-z0-9 ]', '', text)
    else:
        text = re.sub('[^a-z0-9]', '', text)
    return text 

Lectura del archivo de información entregado antes de la carga a la base de datos, como puede observar la columna yr_mo fue cambiada a yrmo.

In [42]:
data= pd.read_csv('datos/Targets.csv', dtype='str')
data.columns = [formatcolumn(val,False) for val in data.columns]
print(data.shape)
data.head()

(15, 3)


,yrmo,metric,target
0,201706,AHT,780
1,201706,CALL_REGEN,0.167
2,201706,TRANSFERS,0.082
3,201706,APPLIED_PER_CALL,0.505
4,201706,BREAKAGE,0.201


En algunas ocasiones, es necesario limpiar carácteres adicionales como retorno de línea (enter)

In [43]:
# Limpieza de otros carácteres especiales
data = data.replace('\n','', regex=True)
data = data.replace('\r','', regex=True)

La carga de datos se realiza con ayuda de la librería de StringIO para manejar el contenido del dataframe en memoria y acelerar el proceso de escritura 

In [45]:
from io import StringIO

output = StringIO()
data.to_csv(output, sep=';', header=False, index=False, columns=data.columns)
output.getvalue()
# jump to start of stream
output.seek(0)

# Insert df into postgres
connection = connAlchemy.raw_connection()
with connection.cursor() as cursor:
    cursor.copy_from(output, "targets_nb", sep=';', null="NULL", columns=(data.columns))
    connection.commit()

In [16]:
#cur.execute("ROLLBACK")
#https://stackoverflow.com/questions/2979369/databaseerror-current-transaction-is-aborted-commands-ignored-until-end-of-tra

## Uso de SQLAlchemy com ORM (Object Relational Mapper)

Al realizar procesos de extracción o carga de datos, **es conveniente independizar las tareas de procesamiento de la información del tipo de base de datos** con las que se esté trabajando, como hemos visto hasta el momento, la librería `psycopg2` trabaja específicamente con postgresql y sentencias de SQL. Por otro lado SQLAlchemy puede interpretar requerimientos concretos de funciones como `read_sql` de la librería pandas.

Para lograr esta independencia podemos usar el concepto de ORM o  Mapeo objeto-relacional, para usar la lógica de la programación orientada a objetos para convertir la información disponible en una fuente de datos relacional (tipo SQL)

```{note}
Información adicional puede ser consultada en https://docs.sqlalchemy.org/en/14/orm/quickstart.html
```


In [46]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship

Instancia de la clase declarative_base para la construcción de las clases asociadas a cada objeto de la base de datos.

In [47]:
Base = declarative_base()

Definción del objeto Hierarchy con sus elementos para el manejo independiente de la base de datos.

In [48]:

class Hierarchy_nueva(Base):
    __tablename__= "hierarchy_nueva"

    id = Column(Integer, primary_key=True)
    eff_dt = Column(String(30))
    term_dt = Column(String(30))
    agent_id = Column(Integer)
    agent_name = Column(String)
    team_lead_id = Column(Integer)
    team_lead_name = Column(String)
    call_center = Column(String)

    def __repr__(self):
        return f"Hierarchy_nueva(id={self.id!r}, eff_dt={self.eff_dt!r}, term_dt={self.term_dt!r}, \
                agent_id = {self.agent_id!r}, agent_name = {self.agent_name!r} team_lead_id = {self.team_lead_id!r}, \
                team_lead_name = {self.team_lead_name!r}, call_center = {self.call_center!r}"        

Conexión con una base de datos local con en AWS

In [49]:
from venv import create
from sqlalchemy import create_engine
#engine = create_engine("sqlite:///databasesqlite.db", echo=True, future=True)

A partir de las clases creadas para la instancia Base y sus correspondientes elementos, crear la tabla en el código de creación de la tabla (DDL - Data Definition Language)

In [50]:
Base.metadata.create_all(connAlchemy) #conexion previamente creada

Creación de una sesión de conexión con SQLAlchemy

In [51]:
from sqlalchemy.orm import Session

Crear dos agentes dentro de la base datos en la tabla Hierarchy.

In [53]:
with Session(connAlchemy) as session:

    albertanderson = Hierarchy_nueva(
         eff_dt="2017-05-01",
         term_dt="2017-12-31",
         agent_id = 201700121,
         agent_name = "Alberto Anderson",
         team_lead_id = 201700011,
         team_lead_name = "Aceona Winnerson",
         call_center = "Williamburg" 
     )
    bartbinnig = Hierarchy_nueva(
         eff_dt="2017-05-01",
         term_dt="2017-12-31",
         agent_id = 201700122,
         agent_name = "Bart Simpson",
         team_lead_id = 201700011,
         team_lead_name = "Aceona Winnerson",
         call_center = "Williamburg" 
     )
    session.add_all([albertanderson, bartbinnig])

    session.commit()

## Conclusiones

Existen diferentes métodos de conexión a una base de datos de tipo SQL, cada una con sus ventajas dependiendo de la implementación que se quiera lograr.

Si la interacción con diferentes motores de bases de datos es constante en los procesos de extracción, transformación y carga, es conveniente implementar soluciones que incluyan la opción de ORM.

Normalmente para conexiones en producción deben involucrarse condiciones adicionales de seguridad dependiendo de la ubicación de la base de datos, protocolos y llaves de seguridad como SSH y archivos .pem.



Guías de Aprendizaje © 2023 by [Rodolfo Meza](https://www.linkedin.com/in/rodolfomeza/) is licensed under [CC BY-NC-SA 4.0](http://creativecommons.org/licenses/by-nc-sa/4.0/?ref=chooser-v1)